### Personal OGS games dump

This notebook generates a .json file of game information for games played with your personal OGS account by calling the OGS API.
It only tracks information *about* each game, and does not dump the games themselves (separate tool incoming).

In [1]:
import requests
import json
import time
import random

Before proceeding, register an OAuth 2.0 application at https://online-go.com/oauth2/applications/.
Copy the client id and client secret strings before saving.
For the other fields I used client type 'public' and authorization type 'password'.

In [2]:
# keys
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"

# used to authorize access to account-specific API calls
username = "YOUR_OGS_USERNAME"
password = "YOUR_OGS_PASSWORD"

In [3]:
# Obtaining authorization token
s = requests.Session()
response = s.post('https://online-go.com/oauth2/token/',data={"grant_type":"password","username":username,"password":password,"client_id":client_id,"client_secret":client_secret})
result = response.json()
token = result['access_token']
headers = {'Authorization': 'Bearer {}'.format(token)}

# Set endpoint, request first page of results
endpoint = "https://online-go.com/api/v1/me/games/"
gamesPage = s.get(endpoint, headers = headers).json()
games = gamesPage['results']
nextPage = gamesPage['next']

The following cell fetches information for the remaining games. The OGS API has a rate limit on requests, so there is also a very primitive rate backoff implemented. You will see some hangups in the output as the rate limit is encountered and the loop has to wait out its backoff time. This code block could take several minutes depending on how many games you've played.

In [4]:
attempts = 0
max_attempts = 10
while nextPage != None: # to avoid timeout, will fix later
    response = s.get(nextPage,  headers = headers)
    # If not rate limited, continue
    if response.status_code != 429:
        attempts = 0
        print(nextPage)
        gamesPage = response.json()
        games.extend(gamesPage['results'])
        nextPage = gamesPage['next']
    # If rate limited, wait and then start again
    else:
        attempts += 1
        if attempts >= max_attempts:
            print("Too many attempts.")
            break
        else:
            time.sleep((2 ** attempts) + random.random())

https://online-go.com/api/v1/me/games/?page=2
https://online-go.com/api/v1/me/games/?page=3
https://online-go.com/api/v1/me/games/?page=4
https://online-go.com/api/v1/me/games/?page=5
https://online-go.com/api/v1/me/games/?page=6
https://online-go.com/api/v1/me/games/?page=7
https://online-go.com/api/v1/me/games/?page=8
https://online-go.com/api/v1/me/games/?page=9
https://online-go.com/api/v1/me/games/?page=10
https://online-go.com/api/v1/me/games/?page=11
https://online-go.com/api/v1/me/games/?page=12
https://online-go.com/api/v1/me/games/?page=13
https://online-go.com/api/v1/me/games/?page=14
https://online-go.com/api/v1/me/games/?page=15
https://online-go.com/api/v1/me/games/?page=16
https://online-go.com/api/v1/me/games/?page=17
https://online-go.com/api/v1/me/games/?page=18
https://online-go.com/api/v1/me/games/?page=19
https://online-go.com/api/v1/me/games/?page=20
https://online-go.com/api/v1/me/games/?page=21
https://online-go.com/api/v1/me/games/?page=22
https://online-go.com

In [ ]:
## Writing list of games to .json file
gamesJson = json.dumps(games, indent = 2)
f = open('games.json', 'w', encoding="utf-8")
f.write(gamesJson)
f.close()